In [ ]:
# This script demonstrates the fluctuations in the B0 field owing to gradient imperfects.
# The script uses the GIRF response measured and estimated on a Siemens Vida Fit 3T Scanner.
#
# Author: dbe@stanford.edu
# Input: mloecher@stanford.edu
#
# Orignally written on May 24, 2025

import matplotlib.pyplot as plt
import numpy as np

npz = np.load('./girf_example/example_GIRF.npz')  # Load the example GIRF
Hz0 = npz['Hz0']  # GTF z->0
Hzz = npz['Hzz']  # GTF z->z

# MRI system specs
sys = {}  # Initialize an empty dictionary
sys['dt'] = 10e-6        # Gradient raster time [s]
sys['gmax'] = 35         # Maximum gradient amplitude [mT/m]
sys['smax'] = 100        # Maximum slewrate [T/m/s]
sys['gamma'] = 42.57e6   # Gyromagnetic ratio [Hz/T]

ggg = PAM_VENC2bipolar(venc, sys)

plt.figure()
plt.plot(ggg)

# Make bipolar gradient
gg = np.hstack([
                np.linspace(0,gmax,int(gmax/smax/dt/1000))[:-1],
                np.linspace(gmax,0,int(gmax/smax/dt/1000))[:-1],
                np.linspace(0,-gmax,int(gmax/smax/dt/1000))[:-1],
                np.linspace(-gmax,0,int(gmax/smax/dt/1000)),
                ])

# Zeropad after by 10ms
N_bipolar = gg.size
N_zeropad = int(10e-3/dt)
gg = np.hstack([gg, np.zeros(N_zeropad)])

plt.figure()
plt.plot(gg)



#PAM_get_grif_field_response

NameError: name 'PAM_VENC2bipolar' is not defined